Problem 1:CSV File encoding existing coder / encryptor

**Encoder**

In [ ]:
import random


def encode_file_usingmap(input_file_name, output_file_name, mapping_file_name):
    """Encode email IDs with unique 64-bit numbers and generate a mapping."""

    # Store used numbers and email mappings
    used_numbers = set()
    user_email_ids = {}

    def get_unique_number():
        """Generate a unique 64-bit number using a while loop to avoid recursion."""
        while True:
            unique_number = random.getrandbits(64)
            if unique_number not in used_numbers:
                used_numbers.add(unique_number)
                return unique_number

    try:
        # Open all files using 'with' for automatic closure
        with open(input_file_name, 'r') as in_file, \
             open(output_file_name, 'w') as out_file, \
             open(mapping_file_name, 'w') as mapping_file:

            # Read header and validate format
            header = in_file.readline().strip()
            expected_header = "user_email_id,product_id,score"

            if header != expected_header:
                print(f"Invalid file format. Expected header: '{expected_header}'.")
                return

            # Write headers for output and mapping files
            out_file.write("user_id,product_id,score\n")
            mapping_file.write("user_id_number,user_email_id\n")

            # Process all lines efficiently
            lines = in_file.readlines()
            for line in lines:
                values = line.strip().split(',')

                # Skip invalid rows that don't match expected format
                if len(values) != 3:
                    print(f"Skipping invalid row: {line.strip()}")
                    continue

                email_id = values[0]

                # Check for existing mapping, otherwise assign a unique number
                if email_id not in user_email_ids:
                    unique_number = get_unique_number()
                    user_email_ids[email_id] = unique_number
                    # Write mapping only for new IDs
                    mapping_file.write(f"{unique_number},{email_id}\n")
                else:
                    unique_number = user_email_ids[email_id]

                # Update the row and write to the encoded file
                values[0] = str(unique_number)
                out_file.write(','.join(values) + "\n")

        print(f"Encoding complete. Encoded data saved to '{output_file_name}' and mapping to '{mapping_file_name}'.")

    except FileNotFoundError:
        print(f"Error: File '{input_file_name}' not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Run the encoding function
if __name__ == "__main__":
    encode_file_usingmap("our_data.csv", "encoded_data.csv", "email_mapping.csv")


**Decoder**

In [ ]:
def decode_file_usingmap(input_file_name, output_file_name, mapping_file_name):
    """Decode encoded user IDs using a mapping and generate the original file."""

    def get_userid_mapping(mapping_file_name):
        """Load user ID to email mapping from the mapping file."""
        u_map = {}
        try:
            with open(mapping_file_name, 'r') as map_file:
                next(map_file)  # Skip header line
                for line in map_file:
                    key, value = line.strip().split(',', 1)
                    u_map[key] = value
        except (FileNotFoundError, ValueError) as e:
            print(f"Error reading mapping file: {e}")
            return None
        return u_map

    # Load the user ID mapping
    user_map = get_userid_mapping(mapping_file_name)
    if not user_map:
        print("Decoding aborted due to missing or invalid mapping.")
        return

    try:
        # Use 'with' to handle file operations safely
        with open(input_file_name, 'r') as in_file, open(output_file_name, 'w') as out_file:
            header = in_file.readline().strip()
            if header != "user_id,product_id,score":
                print(f"Invalid header format in '{input_file_name}'.")
                return

            # Write correct header to output file
            out_file.write("user_email_id,product_id,score\n")

            for line in in_file:
                values = line.strip().split(',')
                # Skip invalid or unmapped rows
                if len(values) == 3 and values[0] in user_map:
                    values[0] = user_map[values[0]]  # Replace with email
                    out_file.write(','.join(values) + "\n")

        print(f"Decoding complete. Output saved to '{output_file_name}'.")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file_name}' not found.")
    except Exception as e:
        print(f"Unexpected error: {e}")


# Run the decoding function
if __name__ == "__main__":
    decode_file_usingmap("encoded_data.csv", "decoded_data.csv", "email_mapping.csv")


Sample input data csv file



```
user_email_id,product_id,score
a@gmail.com,1,4
b@gmail.com,2,3.5
c@gmail.com,3,2.0
```



Encoder file output

```
user_id,product_id,score
17375566958622273556770665904576967,1,4
1735568585959702257666975587866555604576967,2,3.5
17369675969685922616755636604576967,3,2.0
```



Decoder file output



```
user_email_id,product_id,score
a@gmail.com,1,4
b@gmail.com,2,3.5
c@gmail.com,3,2.0
```

